In [1]:
from pycocotools.coco import COCO
from CoOccuranceGraph import CoOccuranceGraph
import json
import cv2

In [ ]:
#toOmit = [1, 183, 105]
toOmit = []
graph = CoOccuranceGraph(toOmit)
toPrint = ['bus', 'car', 'elephant']
print(graph.PositionOfObjects(toPrint))

printXY = graph.PositionOfObjects(toPrint)

print(graph.GetCentroid(printXY))
centroid = graph.GetCentroid(printXY)
for key, value in printXY.items():
    print(key)
    print(graph.GetEuclideanDistance(centroid, value))

print("Euc distance between car and bus")
toTest = ['bus', 'car']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'bus', 'car' = {simDistance}")

print("Euc distance between car and elephant")
toTest = ['car', 'elephant']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'car', 'elephant' = {simDistance}")

print("Euc distance between bus and elephant")
toTest = ['bus', 'elephant']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: bus and elephant = {simDistance}")


print("Euc distance between bed, pillow, tv, remote, carpet")
toTest = ['bed', 'pillow', 'tv', 'remote', 'carpet']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'bed', 'pillow', 'tv', 'remote', 'carpet' = {simDistance}")

print("Euc distance between snowboard, pillow, tv, remote, carpet")
toTest = ['snowboard', 'pillow', 'tv', 'remote', 'carpet']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'snowboard', 'pillow', 'tv', 'remote', 'carpet' = {simDistance}")

# New version that does it all.
print("Euc distance between snowboard, pillow, tv, remote, carpet, Test 2")
toTest = ['snowboard', 'pillow', 'tv', 'remote', 'carpet']
print(f"Sum of Euc Distance for: 'snowboard', 'pillow', 'tv', 'remote', 'carpet' = {graph.FindEuclideanDistance(toTest)}")

print("Tesing if the function does what it should")
toTest = ['bed', 'snowboard', 'pillow', 'tv', 'remote', 'carpet']
a, b ,c ,d = graph.GetLowestAndHighestEucDistance(toTest)
print(a)
print(c)

print(b)
print(d)

In [ ]:
object_annotation_path = "Dataset/annotations/object_annotations/instances_train2017.json"
object_annotations = COCO(annotation_file=object_annotation_path) 

In [ ]:
with open("IdLookup.txt", "r") as f:
    lookup = json.loads(f.read())

In [ ]:
# Creating a map of strings and arrays. Key will be the image id and value is an array of the objects inside the picture.
imagesAndObjects = {}
# Getting all the ids of the images in the coco dataset.
imageIds = object_annotations.getImgIds()
for image in imageIds:
    #Loading the annotations from imageid
    imageAnnotations = object_annotations.loadAnns(object_annotations.getAnnIds(image, iscrowd=None))
    objects = []
    for label in imageAnnotations:
        #If there is a duplicate we do not want to append it.
        if lookup[str(label['category_id'])] in objects:
            continue
        objects.append(lookup[str(label['category_id'])])
        
    imagesAndObjects[str(image)] = objects

In [ ]:
a = 0
b = 0
c = 0
for key, value in imagesAndObjects.items():
    if len(value) == 0:
        a = a +1
    if len(value) == 1:
        b = b +1
    if len(value) == 2:
        c = c +1
print(a)
print(b)
print(c)

In [ ]:
lowHighEuclidian = {}
for key, value in imagesAndObjects.items():
    #If there are less than three unique objects in the image we skip
    if len(value) < 2:
        continue
    low, high, lowExcluded, highExcluded = graph.GetLowestAndHighestEucDistance(value)
    lowHighEuclidian[key] = {'low': low, 'high': high, 'object': value, 'lowexcluded': lowExcluded, 'highexcluded': highExcluded}   

In [ ]:
lowHighEuclidian

In [ ]:
highestDelta = 1.4
ignore = 1.5
highestDict = {}
for key, value in lowHighEuclidian.items():
    delta = value['high'] - value['low']
    if delta > highestDelta and delta < ignore:
        highestDict[key] = [value]
        highestDict['delta'] = delta

In [ ]:
for key, value in highestDict.items():
    #picture = object_annotations.loadAnns(object_annotations.getAnnIds(key, iscrowd=None))
    if key == 'delta':
        continue
    img_info = object_annotations.loadImgs([int(key)])[0]
    img_file_name = img_info["file_name"]
    img_full_path = f"./Dataset/train2017/{img_file_name}"
    
    img = cv2.imread(img_full_path)
    cv2.imshow(f"{value[0]['lowexcluded']}",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()